In [1]:
import sys
import skatePoseModule as spm
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QVBoxLayout, QLineEdit, QLabel, QFileDialog

class MyApp(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()
        self.url = ""
        self.psd = spm.poseDetector(path = self.url)
        
    def initUI(self):        
        qle = QLineEdit(self)
        qle.textChanged[str].connect(self.onChanged)
        
        btn1 = QPushButton('&Button1', self)
        btn1.setCheckable(True)
        btn1.clicked.connect(self.btnRun_clicked)

        btn2 = QPushButton(self)
        btn2.setText('Button&2')

        vbox = QVBoxLayout()
        vbox.addWidget(btn1)
        vbox.addWidget(btn2)

        self.setLayout(vbox)
        self.setWindowTitle('QPushButton')
        self.show()
        
    def onChanged(self, text):
        self.url = text
        
    def btnRun_clicked(self):
        fname=QFileDialog.getOpenFileName(self)        
        self.url = fname[0]
        self.psd = spm.poseDetector(path = self.url)
        print(self.psd.findLandmarks())

if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = MyApp()
    sys.exit(app.exec_())

30


NameError: name 'exit' is not defined

SystemExit: 0

C:\Users\user\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
